In [2]:
import os
import sys
import math
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 1 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def read_file(file):
    data = sio.loadmat(file)
    data = data['data']
    # print(data.shape)
    return data


def compute_DE(signal):
    variance = np.var(signal, ddof=1)
    return math.log(2 * math.pi * math.e * variance) / 2

def decompose(file):
    
    data = read_file(file)
    shape = data.shape
    frequency = 128 #sampling rate

    decomposed_de = np.empty([0, 3, 20])


    for trial in range(40): #use your number of trials

        temp_de = np.empty([0, 20])

        for channel in range(32):
            trial_signal = data[trial, channel, 384:]
            
            theta = butter_bandpass_filter(trial_signal, 4, 8, frequency, order=3)
            alpha = butter_bandpass_filter(trial_signal, 8, 14, frequency, order=3)
            beta = butter_bandpass_filter(trial_signal, 14, 31, frequency, order=3)
            
            DE_theta = np.zeros(shape=[0], dtype=float)
            DE_alpha = np.zeros(shape=[0], dtype=float)
            DE_beta = np.zeros(shape=[0], dtype=float)

            for index in range(20):
                DE_theta = np.append(DE_theta, compute_DE(alpha[index * 3* 128:(index + 1) * 3* 128]))
                DE_alpha = np.append(DE_alpha, compute_DE(alpha[index * 3* 128:(index + 1) * 3* 128]))
                DE_beta = np.append(DE_beta, compute_DE(beta[index * 3*128:(index + 1)* 3 * 128]))
        
            temp_de = np.vstack([temp_de, DE_theta])    
            temp_de = np.vstack([temp_de, DE_alpha])
            temp_de = np.vstack([temp_de, DE_beta])
            
        temp_trial_de = temp_de.reshape(-1, 3, 20)
        decomposed_de = np.vstack([decomposed_de, temp_trial_de])
    print (decomposed_de.shape)
    decomposed_de = decomposed_de.reshape(-1, 32, 3, 20).transpose([0, 3, 2, 1]).reshape(-1, 3, 32)
    decomposed_de = feature_normalize(decomposed_de)
    print("trial_DE shape:", decomposed_de.shape)
    return  decomposed_de

def get_labels(file):
    # 0 valence, 1 arousal
    valence_labels = sio.loadmat(file)["labels"][:, 0] > 5  # valence labels
    arousal_labels = sio.loadmat(file)["labels"][:, 1] > 5  # arousal labels
    final_valence_labels = np.empty([0])
    final_arousal_labels = np.empty([0])
    for i in range(len(valence_labels)):
        for j in range(0, 20):
            final_valence_labels = np.append(final_valence_labels, valence_labels[i])
            final_arousal_labels = np.append(final_arousal_labels, arousal_labels[i])
    print("labels:", final_arousal_labels.shape)
    print(final_valence_labels)
    return final_arousal_labels, final_valence_labels

def wgn(x, snr):
    snr = 10 ** (snr / 10.0)
    xpower = np.sum(x ** 2) / len(x)
    npower = xpower / snr
    return np.random.randn(len(x)) * np.sqrt(npower)

def feature_normalize(data):
    # Reshape the data to a 2D array (samples * bands, features)
    data_reshaped = data.reshape(-1, data.shape[-1])

    # Calculate mean and standard deviation along the last axis (features)
    mean = data_reshaped.mean(axis=-1, keepdims=True)
    sigma = data_reshaped.std(axis=-1, keepdims=True)

    # Apply feature-wise normalization
    data_normalized = (data_reshaped - mean) / sigma

    # Reshape the normalized data back to the original shape
    data_normalized = data_normalized.reshape(data.shape)

    return data_normalized


def feature_normalize(data):
     #Reshape the data to a 1D array
    data_reshaped = data.reshape(-1, 1)
    scaler = MinMaxScaler()
    data_normalized = scaler.fit_transform(data_reshaped).reshape(data.shape)

    return data_normalized

if __name__ == '__main__':
    dataset_dir = "E:/Datasets/DEAP/matlabPREPROCESSED/"

    result_dir = "E:/results/"
    if os.path.isdir(result_dir) == False:
        os.makedirs(result_dir)

        
    for file in os.listdir(dataset_dir):
        print("processing: ", file, "......")
        file_path = os.path.join(dataset_dir, file)
        trial_DE = decompose(file_path)
        arousal_labels, valence_labels = get_labels(file_path)
        sio.savemat(result_dir + "DE3_" + file,
                    {"data": trial_DE, "valence_labels": valence_labels,
                     "arousal_labels": arousal_labels})

processing:  s01.mat ......
(1280, 3, 20)
trial_DE shape: (800, 3, 32)
labels: (800,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1

KeyboardInterrupt: 